<img src="../images/GA-logo.png" style="float: left; margin: 20px; height: 55px">

# Project 2: Singapore Housing Data and Kaggle Challenge

**Primary Learning Objectives:**

1. Creating and iteratively refining a regression model
2. Using Kaggle to practice the modeling process
3. Providing business insights through reporting and presentation.

We will clean the data and build a regression model based on Singapore Housing Dataset to predict the price of a house at sale.

This jupyter notebook focuses on running and evaluating the model used for presentation.

----

## Imports

In [2]:
# import libraries
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, PolynomialFeatures
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

# from copy import deepcopy

import dill

%matplotlib inline

## Functions

In [3]:
# function to get central/outside central based on postal code
def get_central(postal):    
    # mapping
    map_from=list(range(1,83))

    map_to=[1,1,1,1,1,1,2,2,4,4,
            5,5,5,3,3,3,6,7,7,8,
            8,9,9,10,10,10,10,11,11,11,
            12,12,12,13,13,13,13,14,14,14,
            14,15,15,15,15,16,16,16,17,17,
            18,18,19,19,19,20,20,21,21,22,
            22,22,22,22,23,23,23,23,24,24,
            24,25,25,'74',27,27,26,26,28,28,
            17,19]
    
    # take first 2 digits
    #raw_district = postal.astype(str).str.zfill(6)
    #raw_district = raw_district.str.slice(0,2)
    raw_district = postal.astype(int) // 10000
    
    district = raw_district.replace(map_from,map_to)
    
    central = district <= 11
    
    return central

In [4]:
def test_metric(model, X_train, X_test, y_train, y_test, metric='r2_score', cv=5):
    """
    Function to return specified test metric for train, validation and test sets
    
    Parameters
    ----------
       
    model: sklearn model on the outcome variable
        model to run predictions on
    
    X_train: pandas Dataframe
        training predictor dataset
    
    X_test: pandas Dataframe
        test predictor dataset
        
    y_train: pandas Series/Dataframe
        training outcome dataset
        
    y_test: pandas Series/Dataframe
        test outcome dataset

    metric: str
        'r2_score' - evaluates R2 score (default)
        'rmse' - evaluates root mean squared error
        'rmape' - evaluates root mean absolute percentage error
        
    cv: int
        number of cross validation folds (default = 5)
        
    Returns
    -------
    Prints out metric in 3 lines
    
    """
    
    # import libraries
    from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
    from sklearn.model_selection import cross_val_score
    import numpy as np
    
    # if modifier == 'linear':
    #     y_pred_train = model.predict(X_train)
    #     y_pred_test = model.predict(X_test)
    # elif modifier == 'log':
    #     y_pred_train = np.exp(model.predict(X_train))
    #     y_pred_test = np.exp(model.predict(X_test))
    #     y_train = np.exp(y_train)
    #     y_test = np.exp(y_test)
    
    # generate predictions
    #y_pred_train = model.predict(preproc.transform(X_train))
    #y_pred_test = model.predict(preproc.transform(X_test))
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # select function to use
    if metric == 'r2_score':
        train_metric = r2_score(y_train, y_pred_train)
        try:
            cross_metric = cross_val_score(model, X_train, y_train, scoring='r2')
        except: # except clause to catch custom class which does not have 'clone' methods needed for cross_val_score function to work. We write a custom cross_val_score and use this custom function instead
            cross_metric = cross_val_score_custom(model, X_train, y_train, cv=cv, scoring='r2')
        test_metric = r2_score(y_test, y_pred_test)
        
    elif metric == 'rmse':
        train_metric = mean_squared_error(y_train, y_pred_train, squared=False)
        try:
            cross_metric = -1 * cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error')
        except:
            cross_metric = -1 * cross_val_score_custom(model, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error')
        test_metric = mean_squared_error(y_test, y_pred_test, squared=False)
        
    elif metric == 'rmape':
        train_metric = mean_absolute_percentage_error(y_train, y_pred_train)
        try:
            cross_metric = -1 * cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_mean_absolute_percentage_error')
        except:
            cross_metric = -1 * cross_val_score_custom(model, X_train, y_train, cv=cv, scoring='neg_mean_absolute_percentage_error')
        test_metric = mean_absolute_percentage_error(y_test, y_pred_test)
        
    else:
        print('ERROR: Invalid metric chosen')
        return
        
    print(f'Train {metric.upper()}:           \t{train_metric:.4f}')
    print(f'{cv}-Fold CV {metric.upper()}:     \t{np.nanmean(cross_metric):.4f}')
    print(f'Test {metric.upper()}:            \t{test_metric:.4f}')


## Data Import and Processing

In [5]:
# read in clean dataset
housing_df = pd.read_csv('../datasets/train_cleaned.csv')

C:\Users\fishe\AppData\Local\Temp\ipykernel_11076\3481060378.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  housing_df = pd.read_csv('../datasets/train_cleaned.csv')


In [6]:
# create outcome variable y which will be the same for all models
y = housing_df['resale_price']

In [7]:
# impute NIL postal codes first
housing_df.loc[(housing_df['postal'] == 'NIL') & (housing_df['town'] == 'SENGKANG'), 'postal'] = '540000'
housing_df.loc[(housing_df['postal'] == 'NIL') & (housing_df['town'] == 'CHOA CHU KANG'), 'postal'] = '670000'

In [8]:
# convert postal to central districts or outside central districts
housing_df['central_or_not'] = get_central(housing_df['postal'])

In [9]:
# convert mid_storey to 3 categories
housing_df['floor_range'] = ['low' if x <= 15 else 'mid' if x <= 30 else 'high' for x in housing_df['mid_storey']]

In [11]:
# full model variables to be extracted from clean dataset that will be used to different extents by different models
cat_var_list = [
                'planning_area', 
                'flat_type',
                'flat_model',
                'full_flat_type',
                'floor_range',
                'tranc_year', 'tranc_month'
               ]
num_var_list = [
                'hdb_age', 'mid', 'floor_area_sqft', 'central_or_not', 'max_floor_lvl',
                'hawker_nearest_distance',
                'mrt_nearest_distance',
                'mall_nearest_distance', 'longitude', 'latitude'
               ]

X = housing_df[cat_var_list + num_var_list]


In [12]:
# conduct train-test-split for model tuning and evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123, stratify=X[['floor_range', 'central_or_not']])

## Model Construction

In [13]:
# dummy identity scaler to passthrough variables selected without further adjustments
identity_transformer = FunctionTransformer(lambda x: x, feature_names_out='one-to-one')

In [14]:
# create one-hot encoder object
cat_transformer = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')

In [15]:
# create imputer for mall_nearest_distance
imp_transformer = KNNImputer()

### Baseline Model

In [16]:
# baseline model
base_model = DummyRegressor(strategy='mean')

base_model.fit(X_train, y_train)

DummyRegressor()

In [17]:
test_metric(base_model, X_train, X_test, y_train, y_test, metric='rmse')

test_metric(base_model, X_train, X_test, y_train, y_test, metric='rmape')

Train RMSE:           	143416.0302
5-Fold CV RMSE:     	143414.1520
Test RMSE:            	142980.0749
Train RMAPE:           	0.2614
5-Fold CV RMAPE:     	0.2614
Test RMAPE:            	0.2610


### Model A

In [18]:
cat_var_list_E2 = [
    'flat_type',
    'flat_model',
    'tranc_year', 'tranc_month'
]

num_var_list_E2 = [
    'floor_area_sqft', 'central_or_not'
    
]

preprocessor_E2 = Pipeline(
    steps=[
        ('trans', ColumnTransformer(
            transformers=[
                ('cat', cat_transformer, cat_var_list_E2),
                ('dummy', identity_transformer, num_var_list_E2) # identity transform all variables except last one which is the mall_nearest_distance
            ],
            remainder='drop', verbose_feature_names_out=False
            )
        )
    ]
)

model_E2 = Pipeline(
    steps=[
        ('preproc', preprocessor_E2),
        ('lr', LinearRegression())
    ]
)

model_E2.fit(X_train, y_train)

test_metric(model_E2, X_train, X_test, y_train, y_test, metric='rmse')

test_metric(model_E2, X_train, X_test, y_train, y_test, metric='rmape')

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMSE:           	84895.2167
5-Fold CV RMSE:     	84938.0074
Test RMSE:            	84375.2978


C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMAPE:           	0.1379
5-Fold CV RMAPE:     	0.1380
Test RMAPE:            	0.1374


### Model B

In [19]:
cat_var_list_E3 = [
    'flat_type',
    'flat_model',
    'floor_range',
    'tranc_year', 'tranc_month'
]

num_var_list_E3 = [
    'hdb_age','floor_area_sqft', 'central_or_not', 'max_floor_lvl'
    
]

preprocessor_E3 = Pipeline(
    steps=[
        ('trans', ColumnTransformer(
            transformers=[
                ('cat', cat_transformer, cat_var_list_E3),
                ('dummy', identity_transformer, num_var_list_E3) # identity transform all variables except last one which is the mall_nearest_distance
            ],
            remainder='drop', verbose_feature_names_out=False
            )
        )
    ]
)

model_E3 = Pipeline(
    steps=[
        ('preproc', preprocessor_E3),
        ('lr', LinearRegression())
    ]
)

model_E3.fit(X_train, y_train)

test_metric(model_E3, X_train, X_test, y_train, y_test, metric='rmse')

test_metric(model_E3, X_train, X_test, y_train, y_test, metric='rmape')

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMSE:           	73416.4948
5-Fold CV RMSE:     	73454.6081
Test RMSE:            	73229.0758


C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMAPE:           	0.1220
5-Fold CV RMAPE:     	0.1221
Test RMAPE:            	0.1216


### Model C

In [20]:
cat_var_list_E4 = [
    'flat_type',
    'flat_model',
    'floor_range',
    'tranc_year', 'tranc_month'
]

num_var_list_E4 = [
    'hdb_age', 'floor_area_sqft', 'central_or_not', 'max_floor_lvl',
    'hawker_nearest_distance',
    'mrt_nearest_distance'
]

preprocessor_E4 = Pipeline(
    steps=[
        ('trans', ColumnTransformer(
            transformers=[
                ('cat', cat_transformer, cat_var_list_E4),
                ('imp', imp_transformer, ['mall_nearest_distance']),
                ('dummy', identity_transformer, num_var_list_E4) # identity transform all variables except last one which is the mall_nearest_distance
            ],
            remainder='drop', verbose_feature_names_out=False
            )
        )
    ]
)

model_E4 = Pipeline(
    steps=[
        ('preproc', preprocessor_E4),
        ('lr', LinearRegression())
    ]
)

model_E4.fit(X_train, y_train)

test_metric(model_E4, X_train, X_test, y_train, y_test, metric='rmse')

test_metric(model_E4, X_train, X_test, y_train, y_test, metric='rmape')

C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMSE:           	68772.4852
5-Fold CV RMSE:     	68812.4353
Test RMSE:            	68406.7375


C:\ProgramData\Anaconda3\envs\dsi-sg\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Train RMAPE:           	0.1143
5-Fold CV RMAPE:     	0.1143
Test RMAPE:            	0.1136


In [23]:
# Backing out coefficients of variables
model_coef = pd.DataFrame(model_E4['lr'].coef_.transpose(), 
                          index=model_E4['preproc'].get_feature_names_out(),
                          columns=['coefficient']
                         )

model_coef.reset_index(inplace=True)
model_coef.rename(columns={'index':'variable'}, inplace=True)
model_coef.sort_values(by='coefficient', ascending=False, inplace=True)

In [22]:
model_coef

,variable,coefficient
22,flat_model_Terrace,414332.653031
10,flat_model_Improved-Maisonette,143494.544975
24,flat_model_Type S2,121885.556601
19,flat_model_Premium Maisonette,110935.457333
8,flat_model_DBSS,108257.267751
13,flat_model_Model A-Maisonette,94840.943084
50,central_or_not,84449.448514
5,flat_type_MULTI-GENERATION,72687.271936
15,flat_model_Multi Generation,72687.271936
6,flat_model_Adjoined flat,70720.889112
